In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [2]:
import argparse
import sys
import tempfile

In [3]:
import pandas as pd
from six.moves import urllib
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split

In [4]:
# tensorflow version 
print(tf.__version__)

1.12.0


In [5]:
COLUMNS = [
    "count_conscecutive_visits", "recency_day", "recency_week", "visit_number", "output_binary"
]

In [6]:
FEATURE_COLUMNS = [
    "count_conscecutive_visits", "recency_day", "recency_week", "visit_number"
]

In [7]:
INPUT_COLUMNS = [
  # Continuous base columns.
    tf.feature_column.numeric_column('count_conscecutive_visits'),
    tf.feature_column.numeric_column('recency_day'),
    tf.feature_column.numeric_column('recency_week'),
    tf.feature_column.numeric_column('visit_number'),
]

In [8]:
# [START serving-function]
def serving_input_receiver_fn():
    """Build the serving inputs."""
    global FEATURE_COLUMNS
    inputs = {}
    for feat in FEATURE_COLUMNS:
        inputs[feat] = tf.placeholder(shape=[None], dtype=tf.float32)
    
    return tf.estimator.export.ServingInputReceiver(inputs, inputs)
# [END serving-function]

In [9]:
train_file = 'data/train.csv'

In [10]:
df = pd.read_csv(train_file)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 5 columns):
count_conscecutive_visits    300 non-null int64
recency_day                  300 non-null int64
recency_week                 300 non-null int64
visit_number                 300 non-null int64
output_binary                300 non-null int64
dtypes: int64(5)
memory usage: 11.8 KB


In [12]:
y = df["output_binary"]
del df["output_binary"]
X = df

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=12345)

In [14]:
BATCH_SIZE = 40
num_epochs = 1
shuffle = True

In [15]:
train_input_fn = tf.estimator.inputs.pandas_input_fn(
        x=X_train,
        y=y_train,
        batch_size=BATCH_SIZE,
        num_epochs=num_epochs,
        shuffle=shuffle)

In [16]:
test_input_fn = tf.estimator.inputs.pandas_input_fn(
        x=X_test,
        y=y_test,
        batch_size=BATCH_SIZE,
        num_epochs=num_epochs,
        shuffle=shuffle)

In [17]:
classifier = tf.estimator.DNNLinearCombinedClassifier(
              linear_feature_columns=INPUT_COLUMNS,
              dnn_feature_columns=INPUT_COLUMNS,
              dnn_hidden_units=[100, 70, 50, 25])

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/j8/_57p_mmx4576ztxsn0t2d8xw518mys/T/tmpf7w1371l', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1a2ff98a58>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [18]:
serving_fn = serving_input_receiver_fn

In [19]:
export_final = tf.estimator.FinalExporter(
      "export", serving_input_receiver_fn=serving_input_receiver_fn)

In [20]:
train_spec = tf.estimator.TrainSpec(
        input_fn=train_input_fn, max_steps=5)

eval_spec = tf.estimator.EvalSpec(input_fn=test_input_fn,
                                      steps=1,
                                      throttle_secs=1,
                                      start_delay_secs=1)

In [21]:
tf.estimator.train_and_evaluate(classifier, train_spec, eval_spec)

INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps None or save_checkpoints_secs 600.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoints for 0 into /var/folders/j8/_57p_mmx4576ztxsn0t2d8xw518mys/T/tmpf7w1371l/model.ckpt.
INFO:tensorflow:loss = 20.536493, step = 1
INFO:ten

({'accuracy': 0.75,
  'accuracy_baseline': 0.75,
  'auc': 0.3916667,
  'auc_precision_recall': 0.21930124,
  'average_loss': 1.0644901,
  'label/mean': 0.25,
  'loss': 42.579605,
  'precision': 0.0,
  'prediction/mean': 0.21393517,
  'recall': 0.0,
  'global_step': 5},
 [])

In [22]:
classifier.export_savedmodel("export", serving_input_receiver_fn=serving_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['predict']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Signatures EXCLUDED from export because they cannot be be served via TensorFlow Serving APIs:
INFO:tensorflow:'serving_default' : Classification input must be a single string Tensor; got {'count_conscecutive_visits': <tf.Tensor 'Placeholder:0' shape=(?,) dtype=float32>, 'recency_day': <tf.Tensor 'Placeholder_1:0' shape=(?,) dtype=float32>, 'recency_week': <tf.Tensor 'Placeholder_2:0' shape=(?,) dtype=float32>, 'visit_number': <tf.Tensor 'Placeholder_3:0' shape=(?,) dtype=float32>}
INFO:tensorflow:'classification' : Classification input must be a single string Tensor; got {'count_co

b'export/1553839568'

In [23]:
!ls export

1553839568


In [24]:
!saved_model_cli show --dir export/* --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['predict']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['count_conscecutive_visits'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1)
        name: Placeholder:0
    inputs['recency_day'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1)
        name: Placeholder_1:0
    inputs['recency_week'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1)
        name: Placeholder_2:0
    inputs['visit_number'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1)
        name: Placeholder_3:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['class_ids'] tensor_info:
        dtype: DT_INT64
        shape: (-1, 1)
        name: head/predictions/ExpandDims:0
    outputs['classes'] tensor_info:
        dtype: DT_STRING
        shape: (-1, 1)
        name: head/predictions/str_classes:0
    outputs['logistic'] tensor_info:

In [26]:
predict_fn = tf.contrib.predictor.from_saved_model("export/1553839568")

INFO:tensorflow:Restoring parameters from export/1553839568/variables/variables
